In [1]:
from dotenv import load_dotenv
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
#InMemoryChatMessageHistory는 메시지 내에서 메시지를 리스트 형태로 보관함
#애플리케이션 종료시 대화내용이 사라지므로, 계속 보관하고 싶으면 파일이나 DB에 저장해야함
from langchain_core.runnables.history import RunnableWithMessageHistory
#RunnableWithMessageHistory: 모델을 생성할 때 대화 기록을 함께 전달할 수 있도록 하는 클래스스
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv()

model = ChatOpenAI(model = "gpt-4o-mini")

store={}
#session_id를 기준으로 사용자의 대화 기록을 저장하는 딕셔너리 정의

#세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    #만약 해당 세션 ID가 store에 없으면, 새로운 대화 객체를 만들어 저장함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
        #메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id] #해당 세션의 대화 기록을 반환함

#모델 실행시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(model, get_session_history)



In [3]:
#ChatOpenAI를 바로 사용하는 model.invoke() 대신에 with_message_history()를 실행함.

#session_id를 랭체인에서 정한 양식에 맞게 설정함
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 한진규야.")],
    config=config,
)

print(response.content)

안녕하세요, 한진규님! 어떻게 도와드릴까요?


In [4]:

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 한진규입니다. 더 궁금한 점이 있으신가요?


In [7]:
#session id 하나를 계속 사용하다 바꾸면 이전 session id에 기록했던 대화 내용은 현재 기억하지 못함
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

죄송하지만, 당신의 이름을 알 수 없습니다. 당신의 이름을 알려주시면 좋겠어요!


In [8]:
#session id를 이전 것으로 바꾸면 다시 이전 대화 내용을 기억함
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 한진규입니다. 추가로 궁금한 점이 있으면 말씀해 주세요!


In [10]:
#스트림 방식으로 출력하기 - LangChain의 stream을 활용하면 간단히 구현할 수 있음
config = {"configurable": {"session_id": "abc2"}}

for r in with_message_history.stream(
    [HumanMessage(content="내 이름이 뭐라고?")],
    config=config,
):
    print(r.content, end="|")




|당|신|의| 이름|은| 한|진|규|입니다|.| 다른| 질문|이| 있으|시면| 언제|든|지| 말씀|해| 주세요|!||||